In [26]:
#pip install unstructured libmagic python-magic python-magic-bin
#pip install typing-extensions==4.5.0
#pip install streamlit
#pip install faiss-cpu

  Using cached faiss_cpu-1.7.4-cp39-cp39-win_amd64.whl (10.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from llama_index import VectorStoreIndex

In [19]:
#load openAI api key
OPENAI_API_KEY="your openai api key"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [20]:
urls = [
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023",
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023",
]

In [21]:
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [27]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)


In [36]:
# Initialise LLM with required params
llm = OpenAI(temperature=0, max_tokens=500) 

In [37]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openai.as_retriever())
#chain

In [38]:
query = "what is the document about"
# query = "what are the main features of punch iCNG?"


chain({"question": query}, return_only_outputs=True)

{'answer': ' The document is about mobilization and force generation efforts by the Russian government, including attempts to extend social benefits to volunteer formations serving in Ukraine, drafting a bill to include provisions against discrediting volunteer detachments, and efforts to more formally recognize volunteer formations.\n',
 'sources': 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023'}